# Nash Equilibria Computation

#### Homework 2 - Tsatsral Mendsuren, Didier Merk

In this notebook we will give a short demonstration of the program we have in python, which computes all (mixed and pure) Nash Equilibria of a given normal-form game with two players and two actions per player. The program works in all cases where there is a finite amount of Nash Equilibria. In the report of the homework a brief discussion is included, describing what would be required to turn this into a general solution.

In [12]:
# importing the correct packages
import numpy as np
from fractions import Fraction

#### The normal-form game

The template code in this notebook can be used to calculate all Nash equilibria for any normal-form game consisting of two players and two actions for each player. In this report, as an example, we will make use of the normal-form game shown in figure 1. 

To input a normal-form game we make use of two numpy arrays ```u_1``` and ```u_2```: each matrix contains the utilities of a specific player for all possible action profiles.

<figure>
    <img src="notebook_equilibrium.png" width="20%" height="20%" alt="This is an alt">
    <center><figcaption>Figure 1: Two player normal-form game</figcaption></center>
</figure>



In [13]:
# Utility matrices for both players (top-left, top-right, bottom-left, bottom-right)

# player 1 (row player)
u_1 = np.array([[-10, 3], [0, -2]])
# u_1 = np.array([[5, 6], [2, 9]]) # example a in report
# u_1 = np.array([[1, 2], [2, 1]]) # example b in report
# u_1 = np.array([[2, 4], [4, 6]]) # example c in report
# u_1 = np.array([[4, 6], [6, 1]]) # example d in report

# player 2 (column player)
u_2 = np.array([[-10, 0], [3, -2]])
# u_2 = np.array([[9, 3], [2, 5]]) # example a in report
# u_2 = np.array([[2, 1], [1, 2]]) # example b in report
# u_2 = np.array([[2, 4], [4, 6]]) # example c in report
# u_2 = np.array([[7, 3], [3, 3]]) # example d in report

#### Pure Nash equilibria

Pure Nash equilibria occur when no player has an incentive to unilaterally deviate from her action, given the actions of the other players. Intuitively, in our two player normal-form game this means that given the action of the other player, a player would not change his action. Looking back at figure 1 for example, assuming that the <span style="color:darkcyan">column</span> player plays action <span style="color:darkcyan">S</span>, we know that the <span style="color:darkred">row</span> player does not want to swap from his action when it plays <span style="color:darkred">D</span>, because swapping would lower their utility.

In our two player-normal form game, we can therefore check whether players have an incentive to unilaterally deviate from their assigned strategy. This can be done by checking for the <span style="color:darkred">row</span> player if the utility of a certain action profile is at least as high or higher than the other utilities in its column (since we are given the action of the <span style="color:darkcyan">column</span> player). For the <span style="color:darkcyan">column</span> player we can check if the utility of a certain action profile is at least as high or higher than the other utilities in its row (since we are given the action of the <span style="color:darkred">row</span> player).

If, for a specific action profile, it is the case that no player wants to unilaterally switch, we have found a pure Nash equilibrium. The function below does exactly this, and outputs a list containing all Nash equilibria found in a two-player normal-form game. Each element of the list looks as follows (<span style="color:darkred">p</span>, <span style="color:darkcyan">q</span>), where <span style="color:darkred">p</span> indicates the probability of the first player, playing its first action (<span style="color:darkred">D</span> in our example); <span style="color:darkcyan">q</span> indicates the probability of the second player, playing its first action (also <span style="color:darkcyan">D</span> in our example).

In [14]:
# Function that generates a list of all pure Nash equilibria

def pure_NEs(u_1, u_2):
    # retrieve the size of the game
    game_size = u_1.shape
    
    # initialize a list to store pure NEs in
    pure_NE_list = []
    
    # loop over all the cells in the two player normal form game
    for row in range(game_size[0]):
        for col in range(game_size[1]):  
            # when both players DONT want to unilaterally switch
            if u_2[row][col] == u_2[row].max() and u_1[row][col] == u_1[:, col].max():
                # append the found NE
                pure_NE_list.append((1 - row, 1 - col)) 
                
    return pure_NE_list

#### Mixed Nash equilibria

In addition to checking for pure Nash equilibria, we also need to check the existence of mixed Nash equilibria. In mixed strategies, it is now possible for players to play any action with a certain probability. When no player wants to unilaterally deviate from their mixed strategy, we have found a mixed Nash equilibrium.

To find these mixed Nash equilibria we want to find the strategies in which each player is indifferent about their actions. We can calculate this using the formula on slide 11 of lecture 2, where we find the values for <span style="color:darkred">p</span> and <span style="color:darkcyan">q</span> that cause the same expected utility between actions for each player:

<figure>
    <img src="mixed_equilibrium.png" width="20%" height="20%" alt="This is an alt">
    <center><figcaption>Figure 2: Two player normal-form game</figcaption></center>
</figure>

Take the normal-form game in figure 2 above. For player 2, the <span style="color:darkcyan">column</span> player, to be indifferent the expected utilities of both actions should be equal. We can find the value of <span style="color:darkred">p</span> as follows:

$$ a \cdot p + c \cdot (1 - p) = b \cdot p + d \cdot (1 - p) \\
a \cdot p - c \cdot p + c = b \cdot p - d \cdot p + d \\
(a - c) \cdot p + c = (b - d) \cdot p + d \\ 
(a - b - c + d) \cdot p = d - c \\
p = \frac{d - c}{a - b - c + d} $$

In a similar way we can find the value of <span style="color:darkcyan">q</span>, for the <span style="color:darkred">row</span> player to be indifferent:

$$ q = \frac{h - f}{e - f - g + h} $$

When we find these possible values for <span style="color:darkred">p</span> and <span style="color:darkcyan">q</span>, we find the mixed Nash equilibria. However, there are some strange cases, which occur when the denominator is 0. We can not devide by zero, therefore we need to intuitively understand what is happening. When the denominator equals zero ($ a - b - c + d = 0 $) it means that $a - c = b - d$. There are now three scenarios:

* $a = b$: When a is equal than b, according to our formula above it means that $a = b = c = d$, meaning we have a utility matrix in which all values are the same for a player. In this scenario the player is always indifferent and his strategy can take on any value.
* $a > b$: When a is larger than b, it means that c must be larger than d. In this scenario a and c are both larger than b and d, when we look at the normal-form game above, this means that there is a dominant strategy, the <span style="color:darkcyan">column</span> player will always choose to play <span style="color:darkcyan">L</span>, since it always yields higher utility. Therefore q is equal to 1.
* $a < b$: Similar to the statement above, when a is smaller than b, we find that there is a dominant strategy where the <span style="color:darkcyan">column</span> player will always choose to play <span style="color:darkcyan">R</span>, and therefore q is equal to 0.

The final step in this code we have to take is the case where, in the way the code is written, either one of <span style="color:darkred">p</span> and <span style="color:darkcyan">q</span> does not get initialized. This is the case when one player has a denominator that is 0, and the other player does not. This means that a player has a strictly dominant strategy, and the other player will always respond to this, meaning the only Nash equilibria present will be pure.

In [15]:
# Function that finds the mixed Nash equilibrium

def mixed_NEs(u_1, u_2): 
    # player 2 (column player) to be indifferent
    num_p = (u_2[1][1] - u_2[1][0]) #numerator (d - c)
    den_p = (u_2[0][0] - u_2[1][0] - u_2[0][1] + u_2[1][1]) #denominator (a - b - c + d)
    
    # player 1 (row player) to be indifferent
    num_q = (u_1[1][1] - u_1[0][1]) # (h - f)
    den_q = (u_1[0][0] - u_1[1][0] - u_1[0][1] + u_1[1][1]) # (e - f - g + h)
    
    # denominator is 0 for player 2 (column player)
    if den_p == 0:
        if u_2[0][0] == u_2[0][1]:
            q = 'q can be any value ∈ [0, 1]' # does not matter what she does
        elif u_2[0][0] > u_2[0][1]:
            q = 1
        else:
            q = 0
            
    # otherwise p is calculated according to formula
    else:
        p = num_p / den_p 
    
    # denominator is 0 for player 1 (row player)
    if den_q == 0:
        if u_1[0][0] == u_1[1][0]:
            
            p = 'p can be any value ∈ [0, 1]' # does not matter what she does
        elif u_1[0][0] > u_1[1][0]:
            p = 1
        else:
            p = 0
            
    # otherwise q is calculated according to formula
    else:
        q = num_q / den_q
    
    # check if the p and q found are not pure NE
    if 'p' in locals() and 'q' in locals():
        if (p == 0 or p == 1) and (q == 0 or q == 1):
            return None # this means it is a pure NE
        else:
            return p, q # NOT a pure NE
        
    # This means one player had a strictly dominant strategy, meaning no mixed-strategies
    else:
        return None

#### Outputting the results

In the following cell we have written the code to format the output in a readable manner.

In [16]:
# compute the pure NEs
pure_eq = pure_NEs(u_1, u_2)

# compute the mixed NEs
mixed_eq = mixed_NEs(u_1, u_2)

# formatted print statements
print("***********************************")
print("The pure Nash equilibria are:")

if len(pure_eq) == 0:
    print("No pure Nash equilibria found!")
else:
    for i in range(len(pure_eq)):
        print(pure_eq[i])
    
print("")
print("The mixed Nash equilibria are: ")

if mixed_eq is None:
    print("No mixed Nash equilibria found!")
else:
    if type(mixed_eq[0]) == str and type(mixed_eq[1]) != str:
        print(f"({mixed_eq[0]}, {Fraction(mixed_eq[1]).limit_denominator()})")
    elif type(mixed_eq[0]) == str and type(mixed_eq[1]) == str:
        print(f"({mixed_eq[0]}, {mixed_eq[1]})")
    elif type(mixed_eq[0]) != str and type(mixed_eq[1]) == str:
        print(f"({Fraction(mixed_eq[0]).limit_denominator()}, {mixed_eq[1]})")
    else:
        print(f"({Fraction(mixed_eq[0]).limit_denominator()}, {Fraction(mixed_eq[1]).limit_denominator()})")

print("***********************************")

***********************************
The pure Nash equilibria are:
(1, 0)
(0, 1)

The mixed Nash equilibria are: 
(1/3, 1/3)
***********************************
